In [1]:
import os
import shutil
import random
import numpy as np
import pandas as pd

import torch
from torch_geometric.data import Data

## Prediction as inputs

In [2]:
x_pred = []

for i in range(1000):
    pred_path = f'../../res/IEEE_Case118/model_evaluation/PB/pred/pred_{i+1}.csv'
    pred = pd.read_csv(pred_path, header=None, index_col=None)

    x_pred.append(pred.values.copy())

## Ground truth

In [3]:
y = []

for i in range(1000):
    PB_path = f'../../res/IEEE_Case118/model_evaluation/PB/true/true_{i+1}.csv'
    PB = pd.read_csv(PB_path, header=None, index_col=None)

    y.append(PB.values.copy())

## Edge index

In [4]:
# Read branch info
file_path = '../../data/IEEE_Case118/branch_params/branch_params.csv'
branch = pd.read_csv(file_path, header=None, index_col=None).to_numpy()

# Get branch index and attr
edge_index = branch[:, :2] - 1

# Convert to standard format
edge_index = torch.tensor(edge_index.T, dtype=torch.long)

## Create dataset

In [ ]:
# Check the status of old dataset
dir = f'../../train_val_test_dataset/IEEE_Case118/PB_correction/processed'
if not os.path.exists(dir):
    print(f'There is no dataset found!')
else:
    shutil.rmtree(dir)
    print(f'The old dataset has been deleted!')

# Save new dataset
data_list = []
for i in range(len(x_pred)):
    X = torch.from_numpy(x_pred[i]).float()
    Y = torch.from_numpy(y[i]).float()
    graph = Data(x=X, y=Y, edge_index=edge_index)
    data_list.append(graph)

from MyDataset import MyDataset
# Not that if there is already saved dataset, this cell won't work
# Save train, val and test data
root = f'../../train_val_test_dataset/IEEE_Case118/PB_correction/'
MyDataset(root=root, data_list=data_list)